# Adatelőkészítés: 2011-es helyiségnév adatok

A KSH adatai alapján. Letöltés: 2019. május.

In [1]:
import pandas as pd
import numpy as np
import re

## Adatok beimportálása

In [2]:
valasztasi_adatok = pd.read_csv("dataout/valasztasi_eredmenyek.csv")

In [3]:
helyisegnevkonyv = pd.read_excel("datain/Helysegnevkonyv_adattar_2011.xls",
                                skiprows=3,
                                header=None,
                                usecols="A,C,D,G,J:L,N:Z")
helyisegnevkonyv = helyisegnevkonyv.iloc[:-1, :] ## utolsó sor csak összesítés
helyisegnevkonyv.columns = ["község_neve", "község_jogállása", "megye", "kistérség_székhelye", "terület_hektár", "lakónépesség", "lakások_száma"] + \
                        list(map(lambda x: "önkormányzat_"+x, ["bolgár", "cigány", "görög", "horvát", "lengyel", "német", "örmény", "román", "ruszin", "szerb", "szlovák", "szlovén", "ukrán"]))

helyisegnevkonyv.rename(columns = lambda x: x+"_2011", inplace=True)

In [4]:
def to_bool(x):
    if pd.isna(x):
        return False
    else:
        if x:
            return True
        return False
    
## csak 0/1 értékű legyen, hogy van-e nemzetiségi önkormányzat
for c in helyisegnevkonyv.columns:
    if c.startswith("önkormányzat_"):
        helyisegnevkonyv[c] = helyisegnevkonyv[c].map(lambda x: to_bool(x)).astype('int')

In [5]:
helyisegnevkonyv["megye_2011"].where(helyisegnevkonyv["megye_2011"] != "főváros", other='Budapest', inplace=True)

In [6]:
print(sorted(helyisegnevkonyv.megye_2011.unique()) == sorted(valasztasi_adatok.Megye.unique()))

True


Kerületek másként vannak elnevezve, egységesítsük:

In [7]:
helyisegnevkonyv.loc[helyisegnevkonyv["község_neve_2011"].map(lambda x: x.startswith("Budapest")), "község_neve_2011"] = ['Budapest 01',
                                                                                                                     'Budapest 02',
                                                                                                                     'Budapest 03',
                                                                                                                     'Budapest 04',
                                                                                                                     'Budapest 05',
                                                                                                                     'Budapest 06',
                                                                                                                     'Budapest 07',
                                                                                                                     'Budapest 08',
                                                                                                                     'Budapest 09',
                                                                                                                     'Budapest 10',
                                                                                                                     'Budapest 11',
                                                                                                                     'Budapest 12',
                                                                                                                     'Budapest 13',
                                                                                                                     'Budapest 14',
                                                                                                                     'Budapest 15',
                                                                                                                     'Budapest 16',
                                                                                                                     'Budapest 17',
                                                                                                                     'Budapest 18',
                                                                                                                     'Budapest 19',
                                                                                                                     'Budapest 20',
                                                                                                                     'Budapest 21',
                                                                                                                     'Budapest 22',
                                                                                                                     'Budapest 23',] 

In [8]:
helyisegnevkonyv.loc[helyisegnevkonyv["község_neve_2011"].map(lambda x: x.startswith("Budapest")), "község_neve_2011"]

392    Budapest 01
393    Budapest 02
394    Budapest 03
395    Budapest 04
396    Budapest 05
397    Budapest 06
398    Budapest 07
399    Budapest 08
400    Budapest 09
401    Budapest 10
402    Budapest 11
403    Budapest 12
404    Budapest 13
405    Budapest 14
406    Budapest 15
407    Budapest 16
408    Budapest 17
409    Budapest 18
410    Budapest 19
411    Budapest 20
412    Budapest 21
413    Budapest 22
414    Budapest 23
Name: község_neve_2011, dtype: object

In [9]:
print("A választási adatok közt szereplő, a 2011-es helyiségnévadatok közt nem szereplő település:")
[i for i in valasztasi_adatok["Település"] if i not in helyisegnevkonyv["község_neve_2011"].tolist()]

A választási adatok közt szereplő, a 2011-es helyiségnévadatok közt nem szereplő település:


['Balatonakarattya']

Balatonakarattya 2014-ben önállósodott. Sajnos nem találtam 2011-esnél későbbi helyiségnévadattárat.

In [10]:
helyisegnevkonyv["fő_per_hektár"] = helyisegnevkonyv["lakónépesség_2011"] / helyisegnevkonyv["terület_hektár_2011"]

In [11]:
helyisegnevkonyv["fő_per_lakás"] = helyisegnevkonyv["lakónépesség_2011"] / helyisegnevkonyv["lakások_száma_2011"]

In [12]:
helyisegnevkonyv = helyisegnevkonyv.loc[:, [c for c in helyisegnevkonyv if not c.startswith("önkormányzat_")]+[c for c in helyisegnevkonyv if c.startswith("önkormányzat_")]]

In [13]:
pd.concat([helyisegnevkonyv.describe(),
           pd.DataFrame(helyisegnevkonyv.iloc[:, 4:].sum(0), columns=["összesen"]).transpose()])

,terület_hektár_2011,lakónépesség_2011,lakások_száma_2011,fő_per_hektár,fő_per_lakás,önkormányzat_bolgár_2011,önkormányzat_cigány_2011,önkormányzat_görög_2011,önkormányzat_horvát_2011,önkormányzat_lengyel_2011,önkormányzat_német_2011,önkormányzat_örmény_2011,önkormányzat_román_2011,önkormányzat_ruszin_2011,önkormányzat_szerb_2011,önkormányzat_szlovák_2011,önkormányzat_szlovén_2011,önkormányzat_ukrán_2011
count,3.176000e+03,3.176000e+03,3.176000e+03,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000,3176.000000
mean,2.929041e+03,3.144119e+03,1.369318e+03,1.157787,2.301849,0.013224,0.388854,0.011965,0.039673,0.015428,0.133501,0.011965,0.022355,0.023615,0.015113,0.038413,0.003463,0.007242
std,3.559007e+03,1.144670e+04,5.344942e+03,7.795201,0.492356,0.114252,0.487567,0.108744,0.195219,0.123268,0.340169,0.108744,0.147859,0.151869,0.122023,0.192222,0.058759,0.084804
min,5.600000e+01,1.100000e+01,1.300000e+01,0.015777,0.520000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.068000e+03,3.550000e+02,1.600000e+02,0.255473,2.001599,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.867500e+03,8.210000e+02,3.540000e+02,0.419294,2.327982,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.479500e+03,1.953000e+03,7.947500e+02,0.715886,2.614489,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4.879800e+04,2.080160e+05,9.034200e+04,308.377990,5.433735,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
összesen,9.302633e+06,9.985722e+06,4.348955e+06,3677.132294,7310.673026,42.000000,1235.000000,38.000000,126.000000,49.000000,424.000000,38.000000,71.000000,75.000000,48.000000,122.000000,11.000000,23.000000


In [14]:
helyisegnevkonyv[helyisegnevkonyv["fő_per_hektár"] > 200]

,község_neve_2011,község_jogállása_2011,megye_2011,kistérség_székhelye_2011,terület_hektár_2011,lakónépesség_2011,lakások_száma_2011,fő_per_hektár,fő_per_lakás,önkormányzat_bolgár_2011,...,önkormányzat_horvát_2011,önkormányzat_lengyel_2011,önkormányzat_német_2011,önkormányzat_örmény_2011,önkormányzat_román_2011,önkormányzat_ruszin_2011,önkormányzat_szerb_2011,önkormányzat_szlovák_2011,önkormányzat_szlovén_2011,önkormányzat_ukrán_2011
399,Budapest 08,fővárosi kerület,Budapest,Budapest,209,64451,36944,308.37799,1.744559,1,...,1,1,0,1,1,1,1,0,0,0


In [15]:
helyisegnevkonyv.to_csv("dataout/helyisegnevkonyv.csv", index=False)

## Korrelációk

In [16]:
%run definitions.ipynb

In [17]:
korrelaciok = helyisegnevkonyv.corr()
colour_cm(korrelaciok)

Loading BokehJS ...